# Feature Management Platform API in C# with .NET Core

This notebook walks through building a complete Feature Management Platform API using C# and .NET Core, with an in-memory data store, tests, and Docker support.

## 1. Setup Development Environment

First, we need to install and configure the necessary tools for .NET Core development:

1. Install the .NET SDK
2. Install Docker Desktop
3. Install an IDE (Visual Studio or VS Code)

In [ ]:
# Check if .NET SDK is installed
dotnet --version

# Check if Docker is installed
docker --version

## 2. Create a .NET Core Web API Project

Let's create a new .NET Core Web API project and set up the basic project structure with necessary packages.

In [ ]:
# Create a new solution
mkdir FeatureManagementPlatform
cd FeatureManagementPlatform
dotnet new sln -n FeatureManagementPlatform

# Create API project
dotnet new webapi -n FeatureManagementPlatform.Api
dotnet sln add FeatureManagementPlatform.Api

# Create test project
dotnet new xunit -n FeatureManagementPlatform.Tests
dotnet sln add FeatureManagementPlatform.Tests
dotnet add FeatureManagementPlatform.Tests reference FeatureManagementPlatform.Api

# Add required packages
cd FeatureManagementPlatform.Api
dotnet add package Microsoft.AspNetCore.OpenApi
dotnet add package Swashbuckle.AspNetCore

## 3. Define Feature Flag Models

Now, let's define C# classes to represent feature flags, including properties like name, description, enabled status, targeting rules, and variations.

In [ ]:
// Models/FeatureFlag.cs
using System;
using System.Collections.Generic;

namespace FeatureManagementPlatform.Api.Models
{
    public class FeatureFlag
    {
        public Guid Id { get; set; }
        public string Name { get; set; }
        public string Key { get; set; }
        public string Description { get; set; }
        public bool Enabled { get; set; }
        public List<TargetingRule> TargetingRules { get; set; } = new List<TargetingRule>();
        public List<Variation> Variations { get; set; } = new List<Variation>();
        public DateTime CreatedAt { get; set; }
        public DateTime UpdatedAt { get; set; }
    }

    public class TargetingRule
    {
        public Guid Id { get; set; }
        public string Attribute { get; set; }
        public string Operator { get; set; } // equals, contains, startsWith, etc.
        public string Value { get; set; }
        public Guid VariationId { get; set; }
    }

    public class Variation
    {
        public Guid Id { get; set; }
        public string Key { get; set; }
        public string Value { get; set; }
        public int Weight { get; set; } // For percentage rollouts
    }

    public class EvaluationContext
    {
        public string UserId { get; set; }
        public Dictionary<string, string> Attributes { get; set; } = new Dictionary<string, string>();
    }
}

## 4. Implement In-Memory Data Store

Let's create an in-memory repository for feature flags with example data, and design an interface that can be later implemented for a durable data store.

In [ ]:
// Interfaces/IFeatureFlagRepository.cs
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using FeatureManagementPlatform.Api.Models;

namespace FeatureManagementPlatform.Api.Interfaces
{
    public interface IFeatureFlagRepository
    {
        Task<IEnumerable<FeatureFlag>> GetAllAsync();
        Task<FeatureFlag> GetByIdAsync(Guid id);
        Task<FeatureFlag> GetByKeyAsync(string key);
        Task<FeatureFlag> CreateAsync(FeatureFlag featureFlag);
        Task<FeatureFlag> UpdateAsync(FeatureFlag featureFlag);
        Task<bool> DeleteAsync(Guid id);
    }
}

In [ ]:
// Data/InMemoryFeatureFlagRepository.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using FeatureManagementPlatform.Api.Interfaces;
using FeatureManagementPlatform.Api.Models;

namespace FeatureManagementPlatform.Api.Data
{
    public class InMemoryFeatureFlagRepository : IFeatureFlagRepository
    {
        private readonly List<FeatureFlag> _featureFlags;

        public InMemoryFeatureFlagRepository()
        {
            // Initialize with some sample data
            _featureFlags = new List<FeatureFlag>
            {
                new FeatureFlag
                {
                    Id = Guid.NewGuid(),
                    Name = "Dark Mode",
                    Key = "dark-mode",
                    Description = "Enable dark mode for the UI",
                    Enabled = true,
                    CreatedAt = DateTime.UtcNow,
                    UpdatedAt = DateTime.UtcNow,
                    Variations = new List<Variation>
                    {
                        new Variation { Id = Guid.NewGuid(), Key = "on", Value = "true", Weight = 50 },
                        new Variation { Id = Guid.NewGuid(), Key = "off", Value = "false", Weight = 50 }
                    }
                },
                new FeatureFlag
                {
                    Id = Guid.NewGuid(),
                    Name = "New Dashboard",
                    Key = "new-dashboard",
                    Description = "Enable the new dashboard experience",
                    Enabled = false,
                    CreatedAt = DateTime.UtcNow,
                    UpdatedAt = DateTime.UtcNow,
                    TargetingRules = new List<TargetingRule>
                    {
                        new TargetingRule
                        {
                            Id = Guid.NewGuid(),
                            Attribute = "email",
                            Operator = "endsWith",
                            Value = "@company.com"
                        }
                    },
                    Variations = new List<Variation>
                    {
                        new Variation { Id = Guid.NewGuid(), Key = "on", Value = "true", Weight = 100 },
                        new Variation { Id = Guid.NewGuid(), Key = "off", Value = "false", Weight = 0 }
                    }
                }
            };
        }

        public async Task<IEnumerable<FeatureFlag>> GetAllAsync()
        {
            return await Task.FromResult(_featureFlags);
        }

        public async Task<FeatureFlag> GetByIdAsync(Guid id)
        {
            return await Task.FromResult(_featureFlags.FirstOrDefault(f => f.Id == id));
        }

        public async Task<FeatureFlag> GetByKeyAsync(string key)
        {
            return await Task.FromResult(_featureFlags.FirstOrDefault(f => f.Key == key));
        }

        public async Task<FeatureFlag> CreateAsync(FeatureFlag featureFlag)
        {
            featureFlag.Id = Guid.NewGuid();
            featureFlag.CreatedAt = DateTime.UtcNow;
            featureFlag.UpdatedAt = DateTime.UtcNow;
            _featureFlags.Add(featureFlag);
            return await Task.FromResult(featureFlag);
        }

        public async Task<FeatureFlag> UpdateAsync(FeatureFlag featureFlag)
        {
            var existingFlag = _featureFlags.FirstOrDefault(f => f.Id == featureFlag.Id);
            if (existingFlag == null)
                return null;

            var index = _featureFlags.IndexOf(existingFlag);
            featureFlag.UpdatedAt = DateTime.UtcNow;
            _featureFlags[index] = featureFlag;
            return await Task.FromResult(featureFlag);
        }

        public async Task<bool> DeleteAsync(Guid id)
        {
            var featureFlag = _featureFlags.FirstOrDefault(f => f.Id == id);
            if (featureFlag == null)
                return await Task.FromResult(false);

            _featureFlags.Remove(featureFlag);
            return await Task.FromResult(true);
        }
    }
}

## 5. Create API Controllers

Now, let's implement RESTful API controllers with endpoints for creating, retrieving, updating, and deleting feature flags, as well as evaluating flags for specific contexts.

In [ ]:
// Controllers/FeatureFlagsController.cs
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using FeatureManagementPlatform.Api.Interfaces;
using FeatureManagementPlatform.Api.Models;
using FeatureManagementPlatform.Api.Services;
using Microsoft.AspNetCore.Mvc;

namespace FeatureManagementPlatform.Api.Controllers
{
    [ApiController]
    [Route("api/[controller]")]
    public class FeatureFlagsController : ControllerBase
    {
        private readonly IFeatureFlagRepository _repository;
        private readonly IFeatureFlagService _service;

        public FeatureFlagsController(IFeatureFlagRepository repository, IFeatureFlagService service)
        {
            _repository = repository;
            _service = service;
        }

        [HttpGet]
        public async Task<ActionResult<IEnumerable<FeatureFlag>>> GetAll()
        {
            var flags = await _repository.GetAllAsync();
            return Ok(flags);
        }

        [HttpGet("{id}")]
        public async Task<ActionResult<FeatureFlag>> GetById(Guid id)
        {
            var flag = await _repository.GetByIdAsync(id);
            if (flag == null)
                return NotFound();

            return Ok(flag);
        }

        [HttpGet("key/{key}")]
        public async Task<ActionResult<FeatureFlag>> GetByKey(string key)
        {
            var flag = await _repository.GetByKeyAsync(key);
            if (flag == null)
                return NotFound();

            return Ok(flag);
        }

        [HttpPost]
        public async Task<ActionResult<FeatureFlag>> Create(FeatureFlag featureFlag)
        {
            var createdFlag = await _repository.CreateAsync(featureFlag);
            return CreatedAtAction(nameof(GetById), new { id = createdFlag.Id }, createdFlag);
        }

        [HttpPut("{id}")]
        public async Task<ActionResult<FeatureFlag>> Update(Guid id, FeatureFlag featureFlag)
        {
            if (id != featureFlag.Id)
                return BadRequest();

            var updatedFlag = await _repository.UpdateAsync(featureFlag);
            if (updatedFlag == null)
                return NotFound();

            return Ok(updatedFlag);
        }

        [HttpDelete("{id}")]
        public async Task<ActionResult> Delete(Guid id)
        {
            var result = await _repository.DeleteAsync(id);
            if (!result)
                return NotFound();

            return NoContent();
        }

        [HttpPost("evaluate/{key}")]
        public async Task<ActionResult<Variation>> Evaluate(string key, EvaluationContext context)
        {
            var variation = await _service.EvaluateAsync(key, context);
            if (variation == null)
                return NotFound($"Feature flag '{key}' not found or not applicable for the given context");

            return Ok(variation);
        }
    }
}

## 6. Implement Feature Flag Service

Let's create a service layer that implements feature flag evaluation logic, including targeting and rollout strategies based on user attributes.

In [ ]:
// Interfaces/IFeatureFlagService.cs
using System.Threading.Tasks;
using FeatureManagementPlatform.Api.Models;

namespace FeatureManagementPlatform.Api.Services
{
    public interface IFeatureFlagService
    {
        Task<Variation> EvaluateAsync(string flagKey, EvaluationContext context);
    }
}

In [ ]:
// Services/FeatureFlagService.cs
using System;
using System.Linq;
using System.Text.RegularExpressions;
using System.Threading.Tasks;
using FeatureManagementPlatform.Api.Interfaces;
using FeatureManagementPlatform.Api.Models;

namespace FeatureManagementPlatform.Api.Services
{
    public class FeatureFlagService : IFeatureFlagService
    {
        private readonly IFeatureFlagRepository _repository;
        private readonly Random _random = new Random();

        public FeatureFlagService(IFeatureFlagRepository repository)
        {
            _repository = repository;
        }

        public async Task<Variation> EvaluateAsync(string flagKey, EvaluationContext context)
        {
            var flag = await _repository.GetByKeyAsync(flagKey);
            
            if (flag == null || !flag.Enabled)
                return null;

            // Check targeting rules first
            if (flag.TargetingRules.Any())
            {
                foreach (var rule in flag.TargetingRules)
                {
                    if (context.Attributes.TryGetValue(rule.Attribute, out string attributeValue))
                    {
                        if (IsRuleMatch(rule, attributeValue))
                        {
                            return flag.Variations.FirstOrDefault(v => v.Id == rule.VariationId) ?? 
                                   GetRandomVariationByWeight(flag);
                        }
                    }
                }
            }

            // If no targeting rules match or there are no rules, use percentage rollout
            return GetRandomVariationByWeight(flag);
        }

        private bool IsRuleMatch(TargetingRule rule, string attributeValue)
        {
            return rule.Operator.ToLower() switch
            {
                "equals" => attributeValue == rule.Value,
                "contains" => attributeValue.Contains(rule.Value),
                "startswith" => attributeValue.StartsWith(rule.Value),
                "endswith" => attributeValue.EndsWith(rule.Value),
                "matches" => Regex.IsMatch(attributeValue, rule.Value),
                _ => false
            };
        }

        private Variation GetRandomVariationByWeight(FeatureFlag flag)
        {
            int totalWeight = flag.Variations.Sum(v => v.Weight);
            if (totalWeight <= 0)
                return flag.Variations.FirstOrDefault();

            int randomNumber = _random.Next(totalWeight);
            int runningTotal = 0;

            foreach (var variation in flag.Variations)
            {
                runningTotal += variation.Weight;
                if (randomNumber < runningTotal)
                    return variation;
            }

            return flag.Variations.FirstOrDefault();
        }
    }
}

## 7. Configure Dependency Injection

Now, let's set up dependency injection in the Program.cs file to register our services.

In [ ]:
// Program.cs
using FeatureManagementPlatform.Api.Data;
using FeatureManagementPlatform.Api.Interfaces;
using FeatureManagementPlatform.Api.Services;

var builder = WebApplication.CreateBuilder(args);

// Add services to the container.
builder.Services.AddControllers();
builder.Services.AddEndpointsApiExplorer();
builder.Services.AddSwaggerGen();

// Register repository and service
builder.Services.AddSingleton<IFeatureFlagRepository, InMemoryFeatureFlagRepository>();
builder.Services.AddScoped<IFeatureFlagService, FeatureFlagService>();

var app = builder.Build();

// Configure the HTTP request pipeline.
if (app.Environment.IsDevelopment())
{
    app.UseSwagger();
    app.UseSwaggerUI();
}

app.UseHttpsRedirection();
app.UseAuthorization();
app.MapControllers();

app.Run();

## 8. Add Unit and Integration Tests

Let's create a comprehensive test suite using xUnit to test the API endpoints, feature flag evaluation logic, and data access layer.

In [ ]:
// FeatureManagementPlatform.Tests/FeatureFlagServiceTests.cs
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using FeatureManagementPlatform.Api.Data;
using FeatureManagementPlatform.Api.Models;
using FeatureManagementPlatform.Api.Services;
using Xunit;

namespace FeatureManagementPlatform.Tests
{
    public class FeatureFlagServiceTests
    {
        private readonly FeatureFlagService _service;
        private readonly InMemoryFeatureFlagRepository _repository;

        public FeatureFlagServiceTests()
        {
            _repository = new InMemoryFeatureFlagRepository();
            _service = new FeatureFlagService(_repository);
        }

        [Fact]
        public async Task EvaluateAsync_WithValidFlagAndMatchingRule_ReturnsCorrectVariation()
        {
            // Arrange
            var context = new EvaluationContext
            {
                UserId = "user123",
                Attributes = new Dictionary<string, string>
                {
                    { "email", "test@company.com" }
                }
            };

            // Act
            var result = await _service.EvaluateAsync("new-dashboard", context);

            // Assert
            Assert.NotNull(result);
            Assert.Equal("on", result.Key);
            Assert.Equal("true", result.Value);
        }

        [Fact]
        public async Task EvaluateAsync_WithDisabledFlag_ReturnsNull()
        {
            // Arrange
            var flag = await _repository.GetByKeyAsync("new-dashboard");
            flag.Enabled = false;
            await _repository.UpdateAsync(flag);

            var context = new EvaluationContext
            {
                UserId = "user123",
                Attributes = new Dictionary<string, string>
                {
                    { "email", "test@company.com" }
                }
            };

            // Act
            var result = await _service.EvaluateAsync("new-dashboard", context);

            // Assert
            Assert.Null(result);
        }

        [Fact]
        public async Task EvaluateAsync_WithNonExistentFlag_ReturnsNull()
        {
            // Arrange
            var context = new EvaluationContext
            {
                UserId = "user123"
            };

            // Act
            var result = await _service.EvaluateAsync("non-existent-flag", context);

            // Assert
            Assert.Null(result);
        }
    }
}

In [ ]:
// FeatureManagementPlatform.Tests/FeatureFlagRepositoryTests.cs
using System;
using System.Linq;
using System.Threading.Tasks;
using FeatureManagementPlatform.Api.Data;
using FeatureManagementPlatform.Api.Models;
using Xunit;

namespace FeatureManagementPlatform.Tests
{
    public class FeatureFlagRepositoryTests
    {
        private readonly InMemoryFeatureFlagRepository _repository;

        public FeatureFlagRepositoryTests()
        {
            _repository = new InMemoryFeatureFlagRepository();
        }

        [Fact]
        public async Task GetAllAsync_ReturnsAllFlags()
        {
            // Act
            var flags = await _repository.GetAllAsync();

            // Assert
            Assert.NotNull(flags);
            Assert.Equal(2, flags.Count());
        }

        [Fact]
        public async Task GetByKeyAsync_WithValidKey_ReturnsCorrectFlag()
        {
            // Act
            var flag = await _repository.GetByKeyAsync("dark-mode");

            // Assert
            Assert.NotNull(flag);
            Assert.Equal("Dark Mode", flag.Name);
        }

        [Fact]
        public async Task CreateAsync_AddsNewFlag()
        {
            // Arrange
            var newFlag = new FeatureFlag
            {
                Name = "New Feature",
                Key = "new-feature",
                Description = "A new test feature",
                Enabled = true
            };

            // Act
            var result = await _repository.CreateAsync(newFlag);
            var allFlags = await _repository.GetAllAsync();

            // Assert
            Assert.NotNull(result);
            Assert.NotEqual(Guid.Empty, result.Id);
            Assert.Equal(3, allFlags.Count());
            Assert.Contains(allFlags, f => f.Key == "new-feature");
        }

        [Fact]
        public async Task UpdateAsync_WithValidFlag_UpdatesFlag()
        {
            // Arrange
            var flag = await _repository.GetByKeyAsync("dark-mode");
            flag.Description = "Updated description";

            // Act
            var result = await _repository.UpdateAsync(flag);
            var updatedFlag = await _repository.GetByKeyAsync("dark-mode");

            // Assert
            Assert.NotNull(result);
            Assert.Equal("Updated description", updatedFlag.Description);
        }

        [Fact]
        public async Task DeleteAsync_WithValidId_RemovesFlag()
        {
            // Arrange
            var flag = await _repository.GetByKeyAsync("dark-mode");

            // Act
            var result = await _repository.DeleteAsync(flag.Id);
            var allFlags = await _repository.GetAllAsync();

            // Assert
            Assert.True(result);
            Assert.Equal(1, allFlags.Count());
            Assert.DoesNotContain(allFlags, f => f.Key == "dark-mode");
        }
    }
}

## 9. Create Docker Configuration

Now, let's set up Docker files to containerize the API, including a Dockerfile and docker-compose.yml for local development.

In [ ]:
# Dockerfile
FROM mcr.microsoft.com/dotnet/aspnet:7.0 AS base
WORKDIR /app
EXPOSE 80
EXPOSE 443

FROM mcr.microsoft.com/dotnet/sdk:7.0 AS build
WORKDIR /src
COPY ["FeatureManagementPlatform.Api/FeatureManagementPlatform.Api.csproj", "FeatureManagementPlatform.Api/"]
RUN dotnet restore "FeatureManagementPlatform.Api/FeatureManagementPlatform.Api.csproj"
COPY . .
WORKDIR "/src/FeatureManagementPlatform.Api"
RUN dotnet build "FeatureManagementPlatform.Api.csproj" -c Release -o /app/build

FROM build AS publish
RUN dotnet publish "FeatureManagementPlatform.Api.csproj" -c Release -o /app/publish

FROM base AS final
WORKDIR /app
COPY --from=publish /app/publish .
ENTRYPOINT ["dotnet", "FeatureManagementPlatform.Api.dll"]

In [ ]:
# docker-compose.yml
version: '3.8'

services:
  api:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: feature-management-api
    ports:
      - "5000:80"
      - "5001:443"
    environment:
      - ASPNETCORE_ENVIRONMENT=Development
      - ASPNETCORE_URLS=http://+:80
    volumes:
      - ${HOME}/.microsoft/usersecrets:/root/.microsoft/usersecrets:ro
      - ${HOME}/.aspnet/https:/root/.aspnet/https:ro

## 10. Run and Test the API

Finally, let's run the API in a Docker container and test it.

In [ ]:
# Build and run the Docker container
docker-compose build
docker-compose up -d

# Check that the container is running
docker ps

## Testing with curl

Here are some examples of how to interact with the API using curl:

In [ ]:
# Get all feature flags
curl -X GET http://localhost:5000/api/featureflags -H "Content-Type: application/json"

# Get a specific feature flag by key
curl -X GET http://localhost:5000/api/featureflags/key/dark-mode -H "Content-Type: application/json"

# Create a new feature flag
curl -X POST http://localhost:5000/api/featureflags -H "Content-Type: application/json" -d '{
  "name": "Beta Feature",
  "key": "beta-feature",
  "description": "Access to beta features",
  "enabled": true,
  "variations": [
    {
      "key": "on",
      "value": "true",
      "weight": 25
    },
    {
      "key": "off",
      "value": "false",
      "weight": 75
    }
  ]
}'

# Evaluate a feature flag for a specific context
curl -X POST http://localhost:5000/api/featureflags/evaluate/new-dashboard -H "Content-Type: application/json" -d '{
  "userId": "user123",
  "attributes": {
    "email": "test@company.com"
  }
}'

## Conclusion

We've built a complete Feature Management Platform API in C# with .NET Core that includes:

1. Feature flag model with targeting rules and variations
2. In-memory data store with repository pattern
3. RESTful API controllers for CRUD operations
4. Feature evaluation service with targeting and percentage-based rollout
5. Comprehensive unit tests
6. Docker configuration for containerization

This API can now be extended with:
- A persistent database (SQL or NoSQL)
- Authentication and authorization
- Admin UI for managing feature flags
- Client SDKs for different languages/platforms
- Webhooks for feature flag changes
- A/B testing support
- Analytics integration

Happy feature flagging!